# Generic Low Voltage Distribution Network Model

## Overview
This notebook demonstrates the modeling of a typical low voltage (LV) distribution network using PandaPower. The network represents a generic suburban distribution system with multiple feeders serving residential loads.

**Note**: All location-specific information has been anonymized for research and educational purposes.

For more details on the methodology and data used in this model, please refer to the following papers:
C. Panagi et al., "Optimal Operation of Electric Vehicles to Enhance the Flexibility Provision in Active Distribution Grids" 2025, and "Impact of Uncontrolled Electric Vehicle Charging on Unbalanced Suburban Low-Voltage Networks"
Available at: https://sps-lab.org/publication/2025cpanagi/2025CPanagi.pdf, https://sps-lab.org/publication/2025cpanagib/2025CPanagiB.pdf

## Network Topology
The network consists of:
- One HV/MV transformer (11kV/0.4kV)
- Three main feeders (F1, F2, F3) with different configurations
- Multiple load points distributed across the feeders

# <img src="One-Line-Diagram.png" alt="Network Diagram" width="350" style="background:white;">


In [ ]:
### Libraries

import pandapower as pp #import pandapower
import pandas as pd
import math 
import random
import pandapower.networks as nw
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import warnings
import statsmodels.api as sm
from scipy.stats import truncnorm
from datetime import datetime

In [ ]:
# Initialize Network
# Create an empty PandaPower network with base power rating of 315 kVA
net = pp.create_empty_network(sn_mva=0.315)

## Network Components

### Bus Creation
The network buses represent connection points in the power system. We create:
- Main grid bus (11kV)
- Feeder distribution bus (0.4kV) 
- Individual feeder buses for each branch
- Load buses for customer connections

In [ ]:
# Voltage constraints for all buses
min_vm_pu = 0.95  # Minimum voltage limit (95% of nominal)
max_vm_pu = 1.05  # Maximum voltage limit (105% of nominal)

# Initialize feeder bus dictionaries
Feeder1 = {}  # Feeder 1: 24 buses
Feeder2 = {}  # Feeder 2: 8 buses  
Feeder3 = {}  # Feeder 3: 29 buses

# Create main network buses
GridBus = pp.create_bus(net, vn_kv=11, name="Grid-Connection", 
                       min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
FeederBus = pp.create_bus(net, vn_kv=0.4, name="LV-Distribution", 
                         min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")

# Create Feeder 1 buses (24 buses total)
for i in range(0, 24):
    bus_id = str(i)
    vn_kv = 0.4  # Voltage level from Matpower bus data
    bus_name = f"F1-{bus_id}"
    Feeder1[i]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
for i in range(0, 8):
    bus_id = str(i)
    vn_kv = 0.4  # Voltage level from Matpower bus data
    bus_name = f"F2-{bus_id}"
    Feeder2[i]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
for i in range(0, 29):
    bus_id = str(i)
    vn_kv = 0.4  # Voltage level from Matpower bus data
    bus_name = f"F3-{bus_id}"
    Feeder3[i]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")

print(f"Created {len(net.bus)} buses total")
print(f"Feeder 1: {len(Feeder1)} buses")
print(f"Feeder 2: {len(Feeder2)} buses") 
print(f"Feeder 3: {len(Feeder3)} buses")


### External Grid Connection
The external grid represents the upstream network connection point. It provides the slack bus for power flow calculations.

In [ ]:
# Create external grid connection (slack bus)
pp.create_ext_grid(net, GridBus, 
                  vm_pu=1.0,           # Voltage magnitude (p.u.)
                  va_degree=0,         # Voltage angle (degrees)
                  slack_weight=1,      # Slack bus weight
                  s_sc_max_mva=10000,  # Maximum short circuit power (MVA)
                  s_sc_min_mva=8000,   # Minimum short circuit power (MVA)
                  rx_min=0.1,          # Minimum R/X ratio
                  rx_max=0.1,          # Maximum R/X ratio
                  r0x0_max=0.1,        # Maximum zero sequence R/X
                  x0x_max=1.0,         # Maximum zero sequence X/X
                  r0x0_min=0.1,        # Minimum zero sequence R/X
                  x0x_min=1.0)         # Minimum zero sequence X/X

print("External grid connection created successfully")

### Distribution Transformer
The transformer connects the MV grid (11kV) to the LV distribution network (0.4kV). This is a Dyn-type transformer with on-load tap changing capability.

In [ ]:
# trafo1 = pp.create_transformer(net, bus[1], bus[2], name="11kV/0.4kV transformer", std_type="0.63 MVA 10/0.4 kV")
# net.trafo #show transformer table

# net.trafo['vn_hv_kv'][0]=11
# net.trafo['shift_degree'][0]=330

trafo = pp.create_transformer_from_parameters(net, hv_bus=GridBus, lv_bus=FeederBus, 
                                          sn_mva= 0.315, vn_hv_kv= 11, 
                                          vn_lv_kv= 0.4, vk_percent= 6, 
                                          vkr_percent= 0.4, pfe_kw= 0, 
                                          i0_percent= 0, shift_degree= 30, 
                                          tap_side= 'hv',tap_neutral= 0,
                                          tap_min= -2, tap_max= 2,
                                          tap_step_degree= 0,
                                          tap_step_percent= 2.5,
                                          tap_pos=0,
                                          tap_phase_shifter= False,
                                          vk0_percent= 6, vkr0_percent= 0.4, 
                                          mag0_percent= 1, mag0_rx= 0,
                                          si0_hv_partial= 0.9,vector_group= "Dyn",
                                          parallel=1,name="Trasformer"
                                           )

# print(net.trafo)




print("Distribution transformer created successfully")


### Distribution Lines
The network uses different cable types for various sections:
- **Type 1**: Main feeder connections (low impedance)
- **Type 2**: Secondary distribution lines 
- **Type 3**: Three-phase service connections
- **Type 4**: Single-phase service connections

All line parameters are based on standard cable specifications.

In [ ]:
#### LV 300c AI 31/2-C
l1_r_ohm_per_km = 0.0958   # Resistance per km
l1_x_ohm_per_km = 0.07  # Reactance per km
l1_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l1_r0_ohm_per_km = 0.34838   # Resistance per km
l1_x0_ohm_per_km = 1.2328  # Reactance per km
l1_c0_ohm_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l1_max_i_ka = 0.42     # Maximum thermal current rating
l1_length_km = 0.007 

### LV 100mm AI O/H 

l2_r_ohm_per_km = 0.270922   # Resistance per km
l2_x_ohm_per_km = 0.257383  # Reactance per km
l2_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l2_r0_ohm_per_km = 0.554544   # Resistance per km
l2_x0_ohm_per_km = 1.030687  # Reactance per km
l2_c0_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l2_max_i_ka = 0.271     # Maximum thermal current rating
l2_length_km = 0
 
### Service Line 

l3_r_ohm_per_km = 1.273   # Resistance per km
l3_x_ohm_per_km = 0.00001  # Reactance per km
l3_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l3_r0_ohm_per_km = 1.273   # Resistance per km
l3_x0_ohm_per_km = 0.00001  # Reactance per km
l3_c0_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l3_max_i_ka = 0.099     # Maximum thermal current rating
l3_length_km = 0.02  # Using this i will make the p.u to ohms. 


### Main Feeder Bus to Feeder Start Connection ###

pp.create_line_from_parameters(net, FeederBus, Feeder1[0], l1_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

pp.create_line_from_parameters(net, FeederBus, Feeder2[0], l1_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

pp.create_line_from_parameters(net, FeederBus, Feeder3[0], l1_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

## Feeder 1 
#cross
pp.create_line_from_parameters(net, Feeder1[0], Feeder1[1], 23/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[1], Feeder1[2], 6/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[2], Feeder1[3], 18/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[3], Feeder1[4], 11/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[1], Feeder1[5], 24/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[5], Feeder1[6], 19/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[6], Feeder1[7], 23/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[0], Feeder1[8], 4/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[8], Feeder1[9], 8/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[9], Feeder1[10], 22/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[10], Feeder1[11], 17/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[11], Feeder1[12], 19/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[12], Feeder1[13], 21/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[13], Feeder1[14], 26/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#Cross
pp.create_line_from_parameters(net, Feeder1[11], Feeder1[15], 11/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[15], Feeder1[16], 18/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[16], Feeder1[17], 23/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[17], Feeder1[18], 14/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[18], Feeder1[19], 10/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[19], Feeder1[20], 14/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder1[18], Feeder1[21], 11/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[21], Feeder1[22], 10/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[22], Feeder1[23], 26/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )


## Feeder 2
pp.create_line_from_parameters(net, Feeder2[0], Feeder2[1], 18/1000, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

pp.create_line_from_parameters(net, Feeder2[1], Feeder2[2], 15/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[2], Feeder2[3], 18/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[3], Feeder2[4], 13/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
 
pp.create_line_from_parameters(net, Feeder2[4], Feeder2[5], 11/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[5], Feeder2[6], 10/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[6], Feeder2[7], 19/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

## Feeder 3
#cros
pp.create_line_from_parameters(net, Feeder3[0], Feeder3[1], 13/1000, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[0], Feeder3[2], 15/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[2], Feeder3[3], 22/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
 
pp.create_line_from_parameters(net, Feeder3[3], Feeder3[4], 23/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[4], Feeder3[5], 13/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[5], Feeder3[6], 21/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[6], Feeder3[7], 30/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[6], Feeder3[8], 55/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[4], Feeder3[9], 15/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[9], Feeder3[10], 14/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[10], Feeder3[11], 16/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[11], Feeder3[12], 14/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[12], Feeder3[13], 12/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[13], Feeder3[14], 19/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[14], Feeder3[15], 19/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[14], Feeder3[16], 20/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[16], Feeder3[17], 26/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[17], Feeder3[18], 11/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[12], Feeder3[19], 18.5/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[19], Feeder3[20], 6/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[20], Feeder3[21], 11/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[21], Feeder3[22], 17/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[22], Feeder3[23], 19/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[20], Feeder3[24], 13/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[24], Feeder3[25], 20/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[25], Feeder3[26], 21/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[25], Feeder3[27], 24/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[27], Feeder3[28], 15/1000, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

# net.line # show line table

In [ ]:
# Load Bus Creation and Service Line Connections
# Create load buses for customer connections and connect them via service lines

# Initialize load bus dictionaries
Feeder1_load_bus = {}  # 2 loads per Feeder1 node = 48 total loads
Feeder2_load_bus = {}  # 4 loads per Feeder2 node = 32 total loads  
Feeder3_load_bus = {}  # 1 loads per Feeder3 node = 29 total loads

# Feeder 1: Create 3 load buses per distribution node
for i in range(0, 24):
    for j in range(0, 2):
        bus_name = f"F1-Load-{i:02d}-{j:01d}"
        load_bus_idx = i*3 + j
        Feeder1_load_bus[load_bus_idx] = pp.create_bus(net, vn_kv=0.4, name=bus_name,
                                                      min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
        
        # Create service line from distribution node to load bus
        pp.create_line_from_parameters(net, Feeder1[i], Feeder1_load_bus[load_bus_idx], 
                                     l3_length_km, l3_r_ohm_per_km, l3_x_ohm_per_km, 
                                     l3_c_nf_per_km, l3_max_i_ka,
                                     r0_ohm_per_km=l3_r0_ohm_per_km, 
                                     x0_ohm_per_km=l3_x0_ohm_per_km, 
                                     c0_nf_per_km=l3_c0_nf_per_km)

# Feeder 2: Create 6 load buses per distribution node
for i in range(0, 8):
    for j in range(0, 4):
        bus_name = f"F2-Load-{i:02d}-{j:01d}"
        load_bus_idx = i*6 + j
        Feeder2_load_bus[load_bus_idx] = pp.create_bus(net, vn_kv=0.4, name=bus_name,
                                                      min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
        
        # Create service line from distribution node to load bus
        pp.create_line_from_parameters(net, Feeder2[i], Feeder2_load_bus[load_bus_idx], 
                                     l3_length_km, l3_r_ohm_per_km, l3_x_ohm_per_km, 
                                     l3_c_nf_per_km, l3_max_i_ka,
                                     r0_ohm_per_km=l3_r0_ohm_per_km, 
                                     x0_ohm_per_km=l3_x0_ohm_per_km, 
                                     c0_nf_per_km=l3_c0_nf_per_km)

# Feeder 3: Create 2 load buses per distribution node
for i in range(0, 29):
    for j in range(0, 1):
        bus_name = f"F3-Load-{i:02d}-{j:01d}"
        load_bus_idx = i*2 + j
        Feeder3_load_bus[load_bus_idx] = pp.create_bus(net, vn_kv=0.4, name=bus_name,
                                                      min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
        
        # Create service line from distribution node to load bus
        pp.create_line_from_parameters(net, Feeder3[i], Feeder3_load_bus[load_bus_idx], 
                                     l3_length_km, l3_r_ohm_per_km, l3_x_ohm_per_km, 
                                     l3_c_nf_per_km, l3_max_i_ka,
                                     r0_ohm_per_km=l3_r0_ohm_per_km, 
                                     x0_ohm_per_km=l3_x0_ohm_per_km, 
                                     c0_nf_per_km=l3_c0_nf_per_km)

print(f"Created load buses:")
print(f"- Feeder 1: {len(Feeder1_load_bus)} load buses")
print(f"- Feeder 2: {len(Feeder2_load_bus)} load buses")
print(f"- Feeder 3: {len(Feeder3_load_bus)} load buses")
print(f"Total load buses: {len(Feeder1_load_bus) + len(Feeder2_load_bus) + len(Feeder3_load_bus)}")

### Load Modeling
The loads represent residential customers connected to the distribution network. Each load is modeled as an asymmetric three-phase load with realistic power consumption values.


In [ ]:
# Load Parameters (representative of typical residential loads)
# Each load represents a small residential customer
load_p_mw = 0.0004      # Active power per phase (0.4 kW)
load_q_mvar = 0.000132  # Reactive power per phase (0.132 kVAR)
# Power factor ≈ 0.95 (typical for residential loads)

# Create loads for Feeder 1
for bus_idx in Feeder1_load_bus:
    load_name = f"F1-Load-{bus_idx:03d}"
    pp.create_asymmetric_load(net, Feeder1_load_bus[bus_idx], 
                            p_a_mw=load_p_mw, q_a_mvar=load_q_mvar,
                            p_b_mw=load_p_mw, q_b_mvar=load_q_mvar,
                            p_c_mw=load_p_mw, q_c_mvar=load_q_mvar, 
                            name=load_name)

# Create loads for Feeder 2
for bus_idx in Feeder2_load_bus:
    load_name = f"F2-Load-{bus_idx:03d}"
    pp.create_asymmetric_load(net, Feeder2_load_bus[bus_idx], 
                            p_a_mw=load_p_mw, q_a_mvar=load_q_mvar,
                            p_b_mw=load_p_mw, q_b_mvar=load_q_mvar,
                            p_c_mw=load_p_mw, q_c_mvar=load_q_mvar, 
                            name=load_name)

# Create loads for Feeder 3
for bus_idx in Feeder3_load_bus:
    load_name = f"F3-Load-{bus_idx:03d}"
    pp.create_asymmetric_load(net, Feeder3_load_bus[bus_idx], 
                            p_a_mw=load_p_mw, q_a_mvar=load_q_mvar,
                            p_b_mw=load_p_mw, q_b_mvar=load_q_mvar,
                            p_c_mw=load_p_mw, q_c_mvar=load_q_mvar, 
                            name=load_name)

total_loads = len(Feeder1_load_bus) + len(Feeder2_load_bus) + len(Feeder3_load_bus)
total_power_mw = total_loads * 3 * load_p_mw  # 3 phases per load

print(f"Created {total_loads} three-phase loads")
print(f"Total network load: {total_power_mw:.3f} MW")
print(f"Average load per customer: {load_p_mw*1000:.1f} kW per phase")
print("Network model complete - ready for power flow analysis")

## Network Summary

### Network Statistics
- **Total Buses**: 2 (main) + 61 (distribution) + 178 (load) = 241 buses
- **Total Lines**: Main connections + distribution lines + service lines
- **Total Loads**: 178 three-phase residential loads
- **Transformer**: 315 kVA, 11kV/0.4kV, Dyn vector group
- **Network Type**: Unbalanced three-phase distribution system

### Feeder Configuration
- **Feeder 1**: 24 distribution nodes, 72 load points (3 per node)
- **Feeder 2**: 8 distribution nodes, 48 load points (6 per node)  
- **Feeder 3**: 29 distribution nodes, 58 load points (2 per node)

### Usage Instructions
```python
# Run power flow analysis
pp.runpp(net, algorithm='nr', calculate_voltage_angles=True)

# View results
print(net.res_bus)      # Bus voltage results
print(net.res_line)     # Line flow results
print(net.res_trafo)    # Transformer results

# Check for violations
print(f"Voltage violations: {net.res_bus.vm_pu.min():.3f} - {net.res_bus.vm_pu.max():.3f} p.u.")
```

### Network Characteristics
- **Voltage Level**: 400V (0.4kV) distribution
- **Load Type**: Residential (0.4 kW per phase, PF ≈ 0.95)
- **Cable Types**: Standard LV distribution cables
- **Topology**: Radial Network
